In [1]:
# ------------------ IMPORTS ------------------
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import os
import torchmetrics
import wandb
import pandas as pd
import numpy as np
import time
from sklearn.manifold import TSNE
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from tqdm import tqdm
import shutil
import random
from pathlib import Path
from tabulate import tabulate

import torch
import torchvision.models as models
import torchvision.transforms as transforms
from torch import nn

import torch.optim as optim

In [2]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_CLASSES = 4  # change according to your dataset
BATCH_SIZE = 32
EPOCHS = 10
LR = 0.001

print(DEVICE)

cuda


In [3]:
wandb.init(
    project="heira-test",
    config={
        "architecture": "ResNet50",
        "epochs": EPOCHS,
        "batch_size": BATCH_SIZE,
        "learning_rate": LR,
        "optimizer": "Adam"
    }
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jj_12 (eshaan-rithesh2023-vit-chennai). Use `wandb login --relogin` to force relogin
wandb: WARNING Unable to render HTML, can't import display from ipython.core
wandb: WARNING Unable to render HTML, can't import display from ipython.core
wandb: WARNING Unable to render HTML, can't import display from ipython.core
wandb: WARNING Unable to render HTML, can't import display from ipython.core
wandb: WARNING Unable to render HTML, can't import display from ipython.core
wandb: WARNING Unable to render HTML, can't import display from ipython.core


In [4]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])  # ImageNet normalization
])

train_dataset = ImageFolder('data/train', transform=transform)
val_dataset = ImageFolder('data/val', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
print(len(train_dataset.classes))

4


In [5]:
# === DATA LOADERS ===
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],  # ImageNet stats
        std=[0.229, 0.224, 0.225]
    )
])

In [ ]:
train_data = ImageFolder("data/train", transform=transform)
val_data = ImageFolder("data/val", transform=transform)

train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE)

#Model
model = models.resnet50(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

model.fc = nn.Linear(model.fc.in_features, NUM_CLASSES)
model = model.to(DEVICE) #use cuda

# === LOSS & OPTIMIZER ===
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=LR)

/home/joseph/anaconda3/envs/plantLeafDisease/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/joseph/anaconda3/envs/plantLeafDisease/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
print("Labels:", labels)
print("Labels dtype:", labels.dtype)
print("Label unique values:", torch.unique(labels))

NameError: name 'labels' is not defined

In [ ]:
# === WANDB WATCH ===
wandb.watch(model, log="all")

# === EARLY STOPPING ===
best_val_loss = float('inf')
patience_counter = 0

# === TRAINING LOOP ===
for epoch in range(EPOCHS):
    model.train()
    train_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        wandb.log({"batch_loss": loss.item(), "epoch": epoch})

    # === VALIDATION ===
    model.eval()
    val_loss = 0.0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            preds = torch.argmax(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    val_loss /= len(val_loader)
    val_acc = 100 * np.mean(np.array(all_preds) == np.array(all_labels))

    # === LOG TO WANDB ===
    wandb.log({
        "epoch": epoch,
        "train_loss": train_loss / len(train_loader),
        "val_loss": val_loss,
        "val_accuracy": val_acc
    })

    # === CONFUSION MATRIX ===
    cm = confusion_matrix(all_labels, all_preds)
    wandb.log({"confusion_matrix": wandb.plot.confusion_matrix(
        probs=None,
        y_true=all_labels,
        preds=all_preds,
        class_names=class_names
    )})

    # === EARLY STOPPING ===
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        torch.save(model.state_dict(), "best_model.pt")
        wandb.run.summary["best_val_loss"] = best_val_loss
    else:
        patience_counter += 1
        if patience_counter >= PATIENCE:
            print(f"Early stopping triggered at epoch {epoch+1}")
            break

    print(f"Epoch {epoch+1}/{EPOCHS} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")

Epoch 1/10 | Loss: 32.6593 | Val Acc: 93.66%
Epoch 2/10 | Loss: 12.5775 | Val Acc: 96.70%
Epoch 3/10 | Loss: 9.3924 | Val Acc: 97.49%
Epoch 4/10 | Loss: 6.9224 | Val Acc: 97.62%
Epoch 5/10 | Loss: 5.1803 | Val Acc: 97.75%
Epoch 6/10 | Loss: 5.6713 | Val Acc: 97.62%
Epoch 7/10 | Loss: 4.8593 | Val Acc: 98.15%
Epoch 8/10 | Loss: 3.8645 | Val Acc: 98.68%
Epoch 9/10 | Loss: 3.4780 | Val Acc: 98.15%
Epoch 10/10 | Loss: 3.5006 | Val Acc: 98.15%
